# 7g – p‑median optimisation

Solve the **p‑median** location‑allocation problem: choose *P* acutes to act as hubs and assign every spoke to minimise total distance.

In [ ]:

import math, pandas as pd, numpy as np
import pulp
from pathlib import Path

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'
P = 10  # number of hubs to keep open (tune)

R = 6371
def haversine(lat1, lon1, lat2, lon2):
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

hubs = pd.read_csv(ACUTE_CSV)
spokes = pd.concat([pd.read_csv(CDC_CSV), pd.read_csv(CH_CSV)], ignore_index=True)

# Distance matrix
d = pd.DataFrame(index=spokes.index, columns=hubs.index, dtype=float)
for i, spk in spokes.iterrows():
    for j, hub in hubs.iterrows():
        d.loc[i, j] = haversine(spk.latitude, spk.longitude,
                                hub.latitude, hub.longitude)

# Indices
I = spokes.index
J = hubs.index

model = pulp.LpProblem('p‑median', pulp.LpMinimize)
x = pulp.LpVariable.dicts('assign', (I, J), 0, 1, pulp.LpBinary)
y = pulp.LpVariable.dicts('open', J, 0, 1, pulp.LpBinary)

# Objective
model += pulp.lpSum(d.loc[i, j] * x[i][j] for i in I for j in J)

# Constraints
for i in I:
    model += pulp.lpSum(x[i][j] for j in J) == 1
for j in J:
    for i in I:
        model += x[i][j] <= y[j]
model += pulp.lpSum(y[j] for j in J) == P

model.solve(pulp.PULP_CBC_CMD(msg=False))

# Extract assignments
spokes['assigned_hub_id'] = [j for j in J for i in I if pulp.value(x[i][j]) > 0.5]
spokes.head()
